# Trigger-Gaze $\Delta t$
The difference between trigger times and their closest gaze sample, to determine the value of hyperparameter `cnfg.MAX_GAZE_TO_TRIGGER_TIME_DIFF`, which determines if a gaze sample is close enough to a trigger to be considered "co-occurring".

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import config as cnfg

pio.renderers.default = "notebook"      # or "browser"

### Parse Subjects

In [3]:
from analysis.pipeline.preprocess_subjects import preprocess_all_subjects

subjects = preprocess_all_subjects(cnfg.RAW_DATA_PATH, verbose=True)

Preprocessing Subjects: 100%|██████████| 13/13 [02:23<00:00, 11.06s/it]

All 13 subjects processed successfully.


### Calculate $\Delta t$ for each subject-action

In [41]:
action_to_gaze_diffs = dict()
for subj in subjects:
    for tr in subj.get_trials():
        gaze = tr.get_gaze().sort_values("time")
        action = tr.get_actions().sort_values("time")
        for allow_nan_gaze in [True, False]:
            gaze_copy = gaze.copy()
            if not allow_nan_gaze:
                is_missing_left = gaze[['left_x', 'left_y']].isna().any(axis=1)
                is_missing_right = gaze[['right_x', 'right_y']].isna().any(axis=1)
                is_missing = is_missing_left & is_missing_right
                gaze_copy = gaze_copy.loc[~is_missing]
            gaze_times = gaze_copy['time'].values
            action_times = action['time'].values
            diffs = np.abs(gaze_times[:, None] - action_times[None, :])  # shape (gaze, actions)
            min_diff = diffs.min(axis=0)  # minimum difference for each action time; shape (actions,)
            action_to_gaze_diffs[(subj.id, tr.trial_num, allow_nan_gaze)] = min_diff


action_to_gaze_diffs = (
    pd.Series(action_to_gaze_diffs)
    .explode()
    .dropna()
    .reset_index(drop=False)
    .rename(columns={"level_0": "subject", "level_1": "trial", "level_2": "allow_nan_gaze", 0: "time_diff"})
)

action_to_gaze_diffs

,subject,trial,allow_nan_gaze,time_diff
0,15,2,True,1
1,15,2,True,0
2,15,2,False,1
3,15,2,False,0
4,15,6,True,0
...,...,...,...,...
3025,16,58,False,0
3026,16,59,True,0
3027,16,59,False,0
3028,16,60,True,0


In [44]:
action_to_gaze_diffs.groupby(["subject", "trial", "allow_nan_gaze"]).agg(
    min_time_diff=("time_diff", "min"),
    max_time_diff=("time_diff", "max"),
).reset_index()

,subject,trial,allow_nan_gaze,min_time_diff,max_time_diff
0,2,1,False,0,1
1,2,1,True,0,1
2,2,2,False,1,1
3,2,2,True,1,1
4,2,3,False,0,0
...,...,...,...,...,...
1467,22,58,True,0,1
1468,22,59,False,0,1
1469,22,59,True,0,1
1470,22,60,False,1,1


In [47]:
action_to_gaze_diffs.groupby("allow_nan_gaze")["time_diff"].value_counts()

allow_nan_gaze  time_diff
False           0            941
                1            574
True            0            941
                1            574
Name: count, dtype: int64